El proyecto que decidimos realizar consiste en la estimación del valor de viviendas, combinando un modelo de regresión multivariable con una página web interactiva. El objetivo es ofrecer a los usuarios una herramienta fácil de usar que no solo calcule el precio de una propiedad, sino que también brinde una comprensión clara de cómo diversas variables afectan ese valor.

En la página web, los usuarios pueden ingresar parámetros como el tamaño de la vivienda, el número de habitaciones, el estado de la propiedad, entre otros. A través de este input, el modelo estima el precio de la vivienda. Lo que diferencia a nuestra plataforma es que, además de realizar esta estimación, explica detalladamente cómo cada una de estas variables influye en el precio final, permitiendo al usuario entender el impacto que tiene cada factor en el valor de la propiedad.

La interfaz está diseñada para ser interactiva y fácil de usar, permitiendo ajustar estos parámetros y ver cómo varían los resultados en tiempo real. Esta herramienta proporciona a los usuarios una comprensión más profunda de los factores que influyen en el precio de una vivienda, posicionándose como una herramienta valiosa tanto para compradores como para vendedores, ofreciendo transparencia y facilitando una toma de decisiones más informada.

ANTES DE EJECUTAR LA PAGINA WEB SUBIR LAS TRES FOTOS QUE VIENEN ADJUNTAS EN EL ARCHIVO RAR

In [1]:
pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.3 MB/s eta 0:00:00


In [2]:
pip install streamlit-lottie

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 27.2 MB/s eta 0:00:00


In [3]:
pip install pillow

In [4]:
pip install requests

In [5]:
!pip install pandas openpyxl folium streamlit

In [6]:
from google.colab import files
import pandas as pd

# Crear un uploader de archivos
uploaded = files.upload()

# Verificar si se ha subido algún archivo
if uploaded:
    # Obtener el nombre del archivo subido
    filename = list(uploaded.keys())[0]

    # Cargar el archivo CSV en un DataFrame
    df = pd.read_csv(filename)
else:
    print("No se ha subido ningún archivo.")

Saving kc_house_data.csv to kc_house_data.csv


In [26]:
#Esta seccion crea un mapa de calor con la oferta de casas.
import pandas as pd  # Importa la biblioteca pandas para el manejo de datos en formato de tabla.
import folium  # Importa la biblioteca folium para la creación de mapas interactivos.
from folium.plugins import HeatMap  # Importa la función HeatMap de folium para crear mapas de calor.

# Filtra el DataFrame `df` para eliminar filas que tengan valores nulos en las columnas 'lat' o 'long'.
# Esto asegura que solo se utilicen ubicaciones válidas para el mapa.
data = df.dropna(subset=['lat', 'long'])

# Calcula la latitud y longitud promedio de los datos filtrados.
# Esto se usa como punto de inicio para centrar el mapa.
avg_lat = data['lat'].mean()
avg_lon = data['long'].mean()

# Crea un objeto de mapa de folium centrado en la ubicación promedio calculada.
# `zoom_start=10` establece el nivel inicial de zoom del mapa.
mapa = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)

# Convierte las columnas 'lat' y 'long' en una lista de pares de coordenadas para usar en el mapa de calor.
heat_data = data[['lat', 'long']].values.tolist()

# Crea un mapa de calor usando los datos de coordenadas y lo agrega al objeto `mapa`.
HeatMap(heat_data).add_to(mapa)

# Guarda el mapa generado como un archivo HTML.
# El archivo se guardará en la ubicación especificada ('/content/mapa_calor.html').
mapa.save('/content/mapa_calor.html')

In [27]:
%%writefile app.py

import streamlit as st
from Inicio import Mostrar_Inicio  # Importa la función para mostrar la página de inicio.
from Estimador import Mostrar_Estimador  # Importa la función para mostrar la página del estimador.
from Modelo import mostrar_explicacion_modelo  # Importa la función para mostrar la explicación del modelo.
from FAQ import mostrar_faq  # Importa la función para mostrar la sección de preguntas frecuentes.
from ModeloEstimacion import Mostrar_Estimador_Real  # Importa la función para mostrar la estimación real del modelo.
from Buscador import Buscador  # Importa la función para mostrar la página del buscador de viviendas.
from ExplicacionVariablesModelo import analizar_valor  # Importa la función para analizar el valor de las variables del modelo.

# Inicializar el estado de la sesión para almacenar la pestaña seleccionada.
if "selected_tab" not in st.session_state:
    st.session_state.selected_tab = "Inicio"  # Establece "Inicio" como la pestaña predeterminada.

def main():
    # Título principal de la aplicación
    st.title("Estimación de Viviendas")

    # Aplicar estilos personalizados al diseño de la página
    st.markdown(
        """
        <style>
        body {
            background-color: #ADD8E6;  /* Fondo azul claro */
            font-family: Arial, sans-serif;  /* Fuente */
        }
        .stButton>button {
            width: 100%;  /* Botones ocupan todo el ancho de la columna */
            background-color: #4CAF50;  /* Fondo verde */
            color: white;  /* Texto blanco */
            border-radius: 12px;  /* Bordes redondeados */
            font-size: 16px;  /* Tamaño del texto */
        }
        .stButton>button:hover {
            background-color: #45a049;  /* Fondo verde oscuro al pasar el ratón */
        }
        </style>
        """,
        unsafe_allow_html=True,
    )

    # Crear una barra de navegación con botones para cambiar entre pestañas
    col1, col2, col3, col4 = st.columns(4)  # Divide la página en cuatro columnas
    with col1:
        if st.button("Inicio"):  # Botón para la pestaña de inicio
            st.session_state.selected_tab = "Inicio"
    with col2:
        if st.button("Estimador"):  # Botón para la pestaña del estimador
            st.session_state.selected_tab = "Estimador"
    with col3:
        if st.button("Nuestro modelo"):  # Botón para la pestaña del modelo
            st.session_state.selected_tab = "Nuestro modelo"
    with col4:
        if st.button("Preguntas frecuentes"):  # Botón para la pestaña de preguntas frecuentes
            st.session_state.selected_tab = "Preguntas frecuentes"

    # Lógica para mostrar el contenido según la pestaña seleccionada
    tab = st.session_state.selected_tab
    if tab == "Inicio":
        Mostrar_Inicio()  # Llama a la función para mostrar la página de inicio
    elif tab == "Estimador":
        Mostrar_Estimador()  # Llama a la función para mostrar la página del estimador
    elif tab == "Nuestro modelo":
        mostrar_explicacion_modelo()  # Llama a la función para mostrar la explicación del modelo
    elif tab == "Preguntas frecuentes":
        mostrar_faq()  # Llama a la función para mostrar preguntas frecuentes
    elif tab == "Estimador real":
        Mostrar_Estimador_Real()  # Llama a la función para mostrar la estimación real
    elif tab == "Buscar Viviendas":
        Buscador()  # Llama a la función para buscar viviendas
    elif tab == "analizar_valor":
        analizar_valor()  # Llama a la función para analizar el valor de las variables

# Punto de entrada principal de la aplicación
if __name__ == "__main__":
    main()

Overwriting app.py


In [44]:
%%writefile Inicio.py
# Importación de librerías necesarias
import streamlit as st  # Biblioteca principal para crear aplicaciones web interactivas
from streamlit_lottie import st_lottie  # Para integrar animaciones Lottie
from PIL import Image  # Para trabajar con imágenes
import requests  # Para manejar solicitudes HTTP y cargar animaciones desde URLs

# Función para cargar animaciones Lottie desde una URL
def load_lottieurl(url):
    r = requests.get(url)  # Realiza una solicitud HTTP GET a la URL proporcionada
    if r.status_code != 200:  # Verifica si la solicitud fue exitosa
        return None  # Si no, devuelve None
    return r.json()  # Devuelve el contenido JSON de la respuesta

# Función principal que muestra el contenido de la página de inicio
def Mostrar_Inicio():
    # Cargar animación Lottie desde una URL
    lottie_coding = load_lottieurl("https://assets1.lottiefiles.com/packages/lf20_0yfsb3a1.json")

    # Cargar una imagen desde el sistema local
    imagen_inicial = Image.open("/content/Housing.jpg")

    # Contenedor para mostrar la imagen inicial
    with st.container():
        st.image(imagen_inicial, use_container_width=True)  # Imagen ocupa todo el ancho del contenedor

    # Contenedor para el encabezado y descripción del objetivo de la aplicación
    with st.container():
        st.header("Nuestro objetivo")  # Título del bloque
        st.markdown(
            """
            <div class="justify">
           En un mundo incierto, nuestra aplicación ofrece claridad y seguridad para decisiones informadas.
           Diseñada para simplificar el análisis de datos complejos y adaptarse a variables cambiantes,
           ayuda a los usuarios a anticipar y prepararse para el futuro.
            </div>
            """,
            unsafe_allow_html=True  # Permite interpretar HTML en el texto
        )

        # Muestra la animación Lottie en la página
        st_lottie(lottie_coding, height=300, key="coding")

    # Estilo CSS para justificar texto
    st.markdown("""
        <style>
        .justify {
            text-align: justify;
            text-justify: inter-word; /* Mejora la justificación */
        }
        </style>
        """, unsafe_allow_html=True)

    # Título para presentar las opciones que ofrece la página
    st.markdown(
        "<h2 style='text-align: center;'>Qué opciones te damos</h2>",
        unsafe_allow_html=True
    )

    # Bloque para la opción de estimar el valor de una vivienda
    st.header("Estimar el valor de tu vivienda")
    st.markdown(
        """
        <div class="justify">
        Nuestra página te ofrece una herramienta intuitiva para estimar el valor actual de tu vivienda. Al ingresar datos específicos,
        como la ubicación, el tamaño, el estado de la propiedad y otras características relevantes, nuestro modelo utiliza estos
        parámetros para generar una estimación precisa del valor de mercado de tu hogar. Con esta información, podrás tomar decisiones
        informadas sobre tus bienes raíces de manera rápida y confiable.
        </div>
        """,
        unsafe_allow_html=True
    )

    # Botón para navegar a la página del estimador
    if st.button("Ir a Estimar Valor de Casas"):
        st.session_state.selected_tab = "Estimador"  # Actualiza la pestaña seleccionada en el estado de sesión

    # Línea horizontal para separar secciones
    st.markdown("<hr>", unsafe_allow_html=True)

    # Bloque para la opción de buscar una vivienda
    st.header("Buscar una vivienda")
    st.markdown(
        """
        <div class="justify">
        Nuestra página también cuenta con una opción de búsqueda de propiedades, donde puedes explorar viviendas disponibles y conocer sus
        precios en tiempo real. Utilizamos una base de datos actualizada que te permite filtrar por ubicación, tamaño, rango de precios y
        otros criterios para encontrar el hogar que mejor se ajuste a tus necesidades. Con esta herramienta, podrás comparar opciones y
        obtener una visión clara del mercado para tomar decisiones informadas en tu búsqueda de una nueva vivienda.
        </div>
        """,
        unsafe_allow_html=True
    )

    # Botón para navegar a la página del buscador
    if st.button("Ir a Buscar Viviendas"):
        st.session_state.selected_tab = "Buscar Viviendas"  # Actualiza la pestaña seleccionada en el estado de sesión

    # Línea horizontal para separar secciones
    st.markdown("<hr>", unsafe_allow_html=True)

    # Estilo CSS adicional para justificar el texto
    st.markdown("""
        <style>
        .justify {
            text-align: justify;
            text-justify: inter-word;
        }
        </style>
        """, unsafe_allow_html=True)


Overwriting Inicio.py


In [29]:
%%writefile Estimador.py
# Importación de librerías necesarias
import streamlit as st  # Para crear interfaces interactivas
from PIL import Image  # Para trabajar con imágenes

# Cargar la imagen de portada desde una ruta local
imagen_estimador = Image.open("/content/Banner Lane_cover 2.jpg")

# Función principal que muestra el contenido de la página del estimador
def Mostrar_Estimador():
    # Estilo CSS para justificar texto globalmente en la página
    st.markdown("""
        <style>
        .justify {
            text-align: justify; /* Alineación justificada del texto */
            text-justify: inter-word; /* Ajuste adicional de palabras */
        }
        </style>
    """, unsafe_allow_html=True)

    # Contenedor para mostrar la imagen de portada
    with st.container():
        st.image(imagen_estimador, use_container_width=True)  # Imagen ajustada al ancho del contenedor

        # Bloque de introducción al tema
        with st.container():
            st.title("Vende tu hogar con confianza")  # Título principal
            st.markdown(
                """
                <div class="justify">
                Entendemos que la decisión de vender tu hogar puede ser abrumadora. Hay muchas preguntas y factores que pueden influir en
                el valor de tu casa, desde la ubicación y el tamaño hasta el estado de la propiedad y las condiciones del mercado. Nuestra
                plataforma está diseñada para brindarte claridad en este proceso. Te ofrecemos herramientas intuitivas para estimar el
                valor de tu vivienda, teniendo en cuenta todas esas variables cruciales. Con nuestra ayuda, podrás tomar decisiones informadas
                y seguras sobre la venta de tu hogar.
                </div>
                """,
                unsafe_allow_html=True  # Permite la interpretación de HTML en el texto
            )

        # Recordatorio de que el modelo es solo un estimador
        st.header("Recuerda que el resultado solo es un estimador")
        st.markdown(
            """
            <div class="justify">
            Es importante recordar que nuestra herramienta proporciona solo una estimación del valor de tu vivienda. Aunque utilizamos datos
            y modelos avanzados para ofrecerte una valoración precisa, siempre recomendamos consultar a un profesional del sector
            inmobiliario para obtener una evaluación detallada y adaptada a tus necesidades específicas. Tu hogar es una inversión
            significativa, y contar con el asesoramiento adecuado te ayudará a tomar la mejor decisión.
            </div>
            """,
            unsafe_allow_html=True
        )

        # Información sobre cómo contactar a un profesional
        with st.container():
            st.header("¿Cómo me contacto con un profesional?")
            st.write(
                """
                <div class="justify">
                Al buscar un experto para la tasación de tu vivienda, investiga referencias y verifica credenciales.
                Lee reseñas en línea para conocer su reputación y no dudes en solicitar una reunión inicial para
                discutir tus necesidades. Asegúrate de que haya una buena comunicación y que el profesional esté
                disponible para responder tus preguntas.
                </div>
                """,
                unsafe_allow_html=True
            )

        # Introducción a la herramienta de estimación
        st.header("¡Vamos a estimar!")
        st.write(
            """
            <div class="justify">
            ¿Listo para conocer el valor de tu vivienda?
            Ingresa los detalles de tu propiedad y obtén una estimación
            rápida y precisa. ¡Haz clic en el botón a continuación para comenzar!
            </div>
            """,
            unsafe_allow_html=True
        )

        # Añade un espacio adicional para mejorar el diseño
        st.markdown("<br><br>", unsafe_allow_html=True)

        # Botón para redirigir al estimador real
        if st.button("Ir a Estimar Valor de Casas"):
            st.session_state.selected_tab = "Estimador real"  # Cambia la pestaña activa en el estado de sesión


Overwriting Estimador.py


In [30]:
%%writefile Buscador.py

# Importación de librerías necesarias
import pandas as pd  # Para la manipulación de datos, aunque no se usa directamente aquí
import folium  # Para la creación de mapas
from folium.plugins import HeatMap  # Para generar mapas de calor
import streamlit as st  # Para crear interfaces interactivas

# Función principal que muestra el contenido del módulo Buscador
def Buscador():
    # Título principal de la página
    st.title("Existe una enorme oferta de hogares en el área en donde nos estamos enfocando")

    # Texto introductorio sobre el propósito del mapa
    st.write("""
        ¿Quieres hacerte una idea dónde está más concentrada la oferta? Mira el siguiente mapa:
    """)

    # Leer y mostrar el mapa generado previamente como HTML
    with open('/content/mapa_calor.html', 'r') as f:
        mapa_html = f.read()  # Leer el contenido del archivo HTML
    st.components.v1.html(mapa_html, height=500)  # Mostrar el mapa con una altura de 500 píxeles

    # CSS para justificar texto
    st.markdown("""
        <style>
        .justify {
            text-align: justify; /* Alineación del texto justificada */
            text-justify: inter-word; /* Ajuste adicional para palabras */
        }
        </style>
    """, unsafe_allow_html=True)

    # Explicación sobre cómo interpretar el mapa de calor
    st.write("""
        <div class="justify">
        Este es un mapa de calor que visualiza las áreas con mayor concentración de oferta de hogares.
        Las zonas en rojo indican una mayor concentración de propiedades disponibles, lo que sugiere
        que estos lugares tienen una oferta más amplia de viviendas. A medida que la intensidad del
        color disminuye, la concentración de propiedades disponibles en esa área es menor.
        </div>
    """, unsafe_allow_html=True)

# Ejecución de la función si el archivo se ejecuta directamente
if __name__ == "__main__":
    Buscador()

Overwriting Buscador.py


In [31]:
%%writefile ModeloEstimacion.py
import math

# Importación de la librería Streamlit
import streamlit as st

# Función principal para mostrar la herramienta de estimación
def Mostrar_Estimador_Real():
    # Coeficientes del modelo de regresión
    coefficients = {
    'bedrooms': -11518.6788,
    'bathrooms': 2218.0254,
    'sqft_living': 55.8385,
    'sqft_lot': -6.1405,
    'floors': 13260.0,
    'waterfront': 174100.0,
    'view': 32950.0,
    'condition': 27137.0,
    'grade': 51280.0,
    'sqft_above': 38.2422,
    'sqft_basement': 17.5963,
    'yr_built': -134.7760,
    'yr_renovated': 14.0423,
    'sqft_living15': 46.8690,
    'sqft_lot15': -1.9579,
    'house_age': 1760.0985
    }

    # Función para calcular el precio usando los coeficientes y datos de entrada
    def predict_price(input_data):
        price = 0
        for feature, coefficient in coefficients.items():
            price += coefficient * input_data.get(feature, 0)
        return price

    # Interfaz de usuario con Streamlit
    st.title("Estimación del valor de una vivienda")

    # Formulario de ingreso de datos
    with st.form(key='estimacion_form'):
        st.header("Ingresa los datos de la vivienda")

        # Entradas de datos
        sqft_living = st.number_input("Tamaño de la vivienda (sqft)", min_value=0.0)
        bedrooms = st.number_input("Número de habitaciones", min_value=0, max_value=10)
        bathrooms = st.number_input("Número de baños", min_value=0, max_value=10)
        yr_built = st.number_input("Año de construcción", min_value=1900, max_value=2023, value=2020)
        house_age = 2023 - yr_built  # Calculamos la edad de la casa
        grade = st.number_input("Calidad", min_value=0, max_value=10, value=0)
        sqft_lot = st.number_input("Tamaño del terreno (m²)", min_value=0.0)
        floors = st.number_input("Número de pisos", min_value=0, max_value=10)
        waterfront = st.selectbox("¿Está frente al agua?", ["No", "Sí"])
        waterfront = 1 if waterfront == "Sí" else 0
        view = st.number_input("Calificación de la vista (0-4)", min_value=0, max_value=4)
        sqft_above = st.number_input("Metros cuadrados sobre el suelo", min_value=0.0)
        sqft_basement = st.number_input("Metros cuadrados de sótano", min_value=0.0)
        yr_renovated = st.number_input("Año de renovación (0 si no aplica)", min_value=0, max_value=2023)
        sqft_living15 = st.number_input("Tamaño de vivienda promedio en la vecindad (m²)", min_value=0.0)
        sqft_lot15 = st.number_input("Tamaño de terreno promedio en la vecindad (m²)", min_value=0.0)

        # Factores adicionales
        Colegios = st.selectbox("¿Está cerca de colegios?", ["No", "Sí"])
        Colegios = 1 if Colegios == "Sí" else 0
        Carretera = st.selectbox("¿Está próximo a alguna carretera?", ["No", "Sí"])
        Carretera = 1 if Carretera == "Sí" else 0
        Hospital = st.selectbox("¿Está cerca de algún hospital?", ["No", "Sí"])
        Hospital = 1 if Hospital == "Sí" else 0
        Metro = st.selectbox("¿Está cerca de alguna estación de metro?", ["No", "Sí"])
        Metro = 1 if Metro == "Sí" else 0

        # Botón de envío
        submit_button = st.form_submit_button(label='Estimar Valor')

        # Procesar los datos ingresados al hacer clic en el botón
        if submit_button:
            input_data = {
                'sqft_living': sqft_living,
                'bathrooms': bathrooms,
                'bedrooms': bedrooms,
                'sqft_lot': sqft_lot,
                'floors': floors,
                'waterfront': waterfront,
                'view': view,
                'sqft_above': sqft_above,
                'sqft_basement': sqft_basement,
                'yr_renovated': yr_renovated,
                'grade': grade,
                'sqft_living15': sqft_living15,
                'sqft_lot15': sqft_lot15,
                'house_age': house_age,
                'Colegios': Colegios,
                'Carretera': Carretera,
                'Hospital': Hospital,
                'Metro': Metro
            }

            # Aplicar ponderador adicional basado en factores
            PonderadorAdicional = (Colegios + Carretera + Hospital + Metro) / 200  # Ajustado para que sea más balanceado
            valor_estimado = predict_price(input_data) * (1 + PonderadorAdicional)

            # Guardar en el estado de la sesión
            st.session_state.input_data = input_data
            st.session_state.valor_estimado = valor_estimado

            # Mostrar el resultado
            if valor_estimado > 0:
                st.success(f"El valor estimado de la vivienda es: ${valor_estimado:,.2f}")
            else:
                st.error("Error en el cálculo del valor de la vivienda. Por favor, verifica los datos ingresados.")

    # Botón para mostrar análisis adicional
    st.write("Si deseas saber cómo se calculó el valor, haz clic en el siguiente botón:")
    if st.button("Ver Análisis"):
        st.session_state.selected_tab = "analizar_valor"

# Ejecución directa del script
if __name__ == "__main__":
    Mostrar_Estimador_Real()

Overwriting ModeloEstimacion.py


In [47]:
%%writefile ExplicacionVariablesModelo.py
import streamlit as st

# Función principal para analizar y explicar el valor estimado de la vivienda
def analizar_valor():
    # Verifica si los datos de entrada y el valor estimado están en el estado de sesión
    if 'input_data' in st.session_state and 'valor_estimado' in st.session_state:
        input_data = st.session_state.input_data  # Datos de entrada proporcionados por el usuario
        valor_estimado = st.session_state.valor_estimado  # Valor estimado de la vivienda

        # Encabezados principales que muestran el valor estimado y su explicación
        st.header(f"El valor estimado es de la propiedad es de:  ${valor_estimado:,.2f}")
        st.header("Explicación del valor estimado basado en las variables ingresadas:")

        # Explicación basada en la cantidad de habitaciones
        st.subheader("Cantidad de habitaciones")
        if 1 <= input_data['bedrooms'] <= 3:
            st.markdown(f"<p style='text-align: justify;'>Esta vivienda es ideal para una familia pequeña dado que dispone de {input_data['bedrooms']} habitaciones. "
                        "Una casa con menos de 4 habitaciones generalmente tiene un valor accesible para aquellas familias que no requieren mucho espacio adicional.</p>", unsafe_allow_html=True)
        elif input_data['bedrooms'] == 4:
            st.markdown(f"<p style='text-align: justify;'>Esta vivienda es ideal para una familia medianamente grande, ya que dispone de {input_data['bedrooms']} habitaciones. "
                        "Las viviendas con cuatro habitaciones suelen ser atractivas para familias que necesitan más espacio, pero no requieren propiedades excesivamente grandes.</p>", unsafe_allow_html=True)
        elif input_data['bedrooms'] == 5:
            st.markdown(f"<p style='text-align: justify;'>Esta vivienda es adecuada para una familia grande, ya que dispone de {input_data['bedrooms']} habitaciones. "
                        "Las casas con cinco o más habitaciones ofrecen un confort y amplitud adecuados para familias grandes o aquellas que necesitan espacio extra para oficinas o áreas de trabajo.</p>", unsafe_allow_html=True)
        else:
            st.markdown(f"<p style='text-align: justify;'>Esta vivienda es ideal para familias muy grandes, con más de {input_data['bedrooms']} habitaciones. "
                        "Casas con una cantidad de habitaciones tan elevada pueden ser perfectas para grandes familias o para quienes desean tener múltiples áreas de descanso.</p>", unsafe_allow_html=True)

        # Explicación sobre la ubicación frente al agua
        st.subheader("Frente al agua")
        if input_data['waterfront'] == 1:
            st.markdown("<p style='text-align: justify;'>La vivienda tiene una ubicación privilegiada frente al agua, lo cual aporta valor al hogar. "
                        "Las propiedades frente al agua son altamente demandadas, ya que ofrecen vistas espectaculares y una experiencia de vida única. Esto, por lo general, incrementa significativamente el valor de la propiedad.</p>", unsafe_allow_html=True)
        else:
            st.markdown("<p style='text-align: justify;'>La vivienda no tiene vista al agua, lo que puede afectar su valor. "
                        "Las casas que no se encuentran frente al agua pueden ser menos atractivas debido a la falta de vistas privilegiadas, pero aún pueden tener un valor considerable dependiendo de su ubicación y características internas.</p>", unsafe_allow_html=True)

        # Explicación sobre la vista disponible
        st.subheader("Vista")
        if input_data['view'] == 0:
            st.markdown("<p style='text-align: justify;'>La vivienda no tiene vista destacada, lo que puede reducir su valor en comparación con propiedades que sí la tienen. "
                        "Una propiedad sin vistas atractivas generalmente se percibe como menos exclusiva, lo que puede influir negativamente en su precio.</p>", unsafe_allow_html=True)
        elif input_data['view'] == 1:
            st.markdown("<p style='text-align: justify;'>La vivienda tiene una vista moderada, lo que podría hacerla más atractiva pero no tan exclusiva como propiedades con mejores vistas. "
                        "Aunque no sea una vista panorámica, aún puede añadir valor al hogar por el entorno visual que proporciona.</p>", unsafe_allow_html=True)
        elif input_data['view'] == 2:
            st.markdown("<p style='text-align: justify;'>La vivienda tiene una vista decente, lo que añade valor a la propiedad al ofrecer un entorno agradable. "
                        "Una vista decente mejora la calidad de vida, aunque no sea espectacular, puede ser suficiente para incrementar el valor de la propiedad.</p>", unsafe_allow_html=True)
        else:
            st.markdown("<p style='text-align: justify;'>La vivienda tiene una excelente vista, lo que aumenta significativamente su atractivo y valor. "
                        "Las propiedades con vistas excepcionales, como vistas al mar o a la montaña, suelen tener una mayor demanda y, por ende, un valor más alto.</p>", unsafe_allow_html=True)

        # Explicación sobre la edad de la casa
        st.subheader("Edad de la casa")
        if input_data['house_age'] < 5:
            st.markdown(f"<p style='text-align: justify;'>La vivienda es muy nueva (edad: {input_data['house_age']} años), lo que generalmente contribuye a un mayor valor debido a la modernidad y estado de conservación. "
                        "Una casa nueva suele requerir menos mantenimiento y puede contar con diseños y materiales de construcción más actuales, lo que la hace más atractiva para los compradores.</p>", unsafe_allow_html=True)
        elif input_data['house_age'] < 20:
            st.markdown(f"<p style='text-align: justify;'>La vivienda tiene una edad moderada (edad: {input_data['house_age']} años), lo que podría significar que ya está asentada pero no requiere grandes renovaciones. "
                        "Las casas de entre 5 y 20 años suelen ser una excelente opción para quienes buscan un equilibrio entre antigüedad y valor de mercado.</p>", unsafe_allow_html=True)
        elif input_data['house_age'] < 30:
            st.markdown(f"<p style='text-align: justify;'>La vivienda tiene más de 20 años (edad: {input_data['house_age']} años), lo que puede implicar que necesita algunas actualizaciones o mantenimiento. "
                        "Las casas con más de 20 años pueden requerir renovaciones para estar a la altura de las expectativas actuales de los compradores, lo cual puede afectar el valor de la propiedad.</p>", unsafe_allow_html=True)
        else:
            st.markdown(f"<p style='text-align: justify;'>La vivienda tiene una edad considerable (edad: {input_data['house_age']} años), lo que podría afectar su valor debido al desgaste y la posible necesidad de renovaciones. "
                        "Las casas antiguas pueden tener un valor histórico o arquitectónico, pero también podrían necesitar modernización para cumplir con los estándares actuales.</p>", unsafe_allow_html=True)

        # Tamaño de la vivienda
        st.subheader("Tamaño de la vivienda")
        if input_data['sqft_living'] > 3000:
            st.markdown(f"<p style='text-align: justify;'>El tamaño de la vivienda es grande ({input_data['sqft_living']} sqft), lo que contribuye a un mayor valor debido al espacio adicional que ofrece. "
                        "Las casas grandes suelen ser más caras debido al mayor espacio habitable, lo que las hace atractivas para familias grandes o personas que valoran la amplitud.</p>", unsafe_allow_html=True)
        elif input_data['sqft_living'] > 2000:
            st.markdown(f"<p style='text-align: justify;'>El tamaño de la vivienda es moderado-grande ({input_data['sqft_living']} sqft), lo que la hace adecuada para una familia con necesidades de espacio. "
                        "Con más de 2000 pies cuadrados, la propiedad ofrece suficiente espacio para vivir cómodamente, pero sin ser excesiva.</p>", unsafe_allow_html=True)
        elif input_data['sqft_living'] > 1500:
            st.markdown(f"<p style='text-align: justify;'>El tamaño de la vivienda es moderado ({input_data['sqft_living']} sqft), lo que puede ser ideal para una familia pequeña o una pareja. "
                        "Aunque no sea tan grande, aún proporciona suficiente espacio para un estilo de vida cómodo, lo que la hace adecuada para quienes buscan una propiedad más accesible.</p>", unsafe_allow_html=True)
        else:
            st.markdown(f"<p style='text-align: justify;'>El tamaño de la vivienda es pequeño ({input_data['sqft_living']} sqft), lo que podría limitar el valor en comparación con viviendas más grandes. "
                        "Las casas pequeñas pueden ser adecuadas para personas solteras o parejas, pero su tamaño puede restar valor frente a otras propiedades con más espacio.</p>", unsafe_allow_html=True)

        # Metros cuadrados sobre el suelo
        st.subheader("Metros cuadrados sobre el suelo")
        if input_data['sqft_above'] > input_data['sqft_basement']:
            st.markdown("<p style='text-align: justify;'>La vivienda tiene más espacio sobre el suelo que en el sótano, lo que generalmente es preferido por los compradores. "
                        "Las áreas sobre el suelo suelen tener un valor más alto debido a su accesibilidad y visibilidad, mientras que los sótanos a menudo se perciben como menos atractivos.</p>", unsafe_allow_html=True)
        elif input_data['sqft_above'] == input_data['sqft_basement']:
            st.markdown("<p style='text-align: justify;'>La vivienda tiene un espacio equilibrado entre el área sobre el suelo y el sótano, lo que le otorga funcionalidad. "
                        "Este tipo de distribución es útil para quienes desean un espacio adicional, como un área de almacenamiento o un cuarto de juegos, sin comprometer el valor de la propiedad.</p>", unsafe_allow_html=True)
        else:
            st.markdown("<p style='text-align: justify;'>La vivienda tiene más área en el sótano que sobre el suelo, lo que podría ser un punto positivo para algunos compradores pero también puede restar valor en comparación con propiedades sin sótano. "
                        "Aunque un sótano grande puede ser útil para almacenamiento o espacios adicionales, a menudo se valora menos que el área habitable principal de la casa.</p>", unsafe_allow_html=True)

        # Años de renovación
        st.subheader("Años de renovación")
        if input_data['yr_renovated'] > 0:
            st.markdown(f"<p style='text-align: justify;'>La vivienda ha sido renovada recientemente en {input_data['yr_renovated']}, lo que mejora su valor al estar en mejores condiciones y con características modernas. "
                        "Las renovaciones recientes a menudo significan menos trabajo por parte de los nuevos propietarios, lo que aumenta el atractivo y el valor de la propiedad.</p>", unsafe_allow_html=True)
        else:
            st.markdown("<p style='text-align: justify;'>La vivienda no ha sido renovada recientemente, lo que podría afectar su valor si se requieren actualizaciones en sus instalaciones o diseño. "
                        "Una propiedad sin renovaciones recientes puede parecer menos atractiva o tener un valor menor debido a la necesidad de actualizaciones.</p>", unsafe_allow_html=True)

        # Tamaño de terreno
        st.subheader("Tamaño de terreno")
        if input_data['sqft_lot'] > 10000:
            st.markdown(f"<p style='text-align: justify;'>El terreno es bastante grande ({input_data['sqft_lot']} sqft), lo cual agrega valor al hogar debido a su potencial de expansión o uso. "
                        "Un terreno grande ofrece muchas oportunidades para el desarrollo o la expansión de la propiedad, lo que puede aumentar significativamente su valor.</p>", unsafe_allow_html=True)
        elif input_data['sqft_lot'] > 5000:
            st.markdown(f"<p style='text-align: justify;'>El terreno tiene un tamaño adecuado ({input_data['sqft_lot']} sqft), lo que permite un buen uso del espacio sin ser excesivo. "
                        "El tamaño de terreno adecuado puede ser perfecto para quienes buscan equilibrio entre un espacio funcional y el costo de propiedad.</p>", unsafe_allow_html=True)
        else:
            st.markdown(f"<p style='text-align: justify;'>El terreno es pequeño ({input_data['sqft_lot']} sqft), lo que podría limitar el valor en comparación con propiedades en terrenos más grandes. "
                        "Las propiedades en terrenos más pequeños pueden ser menos atractivas si se busca un espacio amplio para jardinería, expansión o privacidad.</p>", unsafe_allow_html=True)

        # Adicionales
        if input_data['Colegios'] == 1:
          st.subheader("Colegios")
          st.markdown("<p style='text-align: justify;'>La vivienda está cerca de colegios, lo que puede aumentar su valor, especialmente para familias con niños. "
                "La proximidad a instituciones educativas es una característica importante para aquellos que tienen hijos, ya que les proporciona comodidad y ahorro de tiempo.</p>", unsafe_allow_html=True)
        if input_data['Carretera'] == 1:
          st.subheader("Carretera")
          st.markdown("<p style='text-align: justify;'>La vivienda está cerca de una carretera, lo que puede aumentar el valor pero también puede traer ruido y contaminación. "
                      "La cercanía a una carretera principal facilita el acceso y la movilidad, pero puede implicar molestias debido al tráfico y los ruidos, lo que influye en la preferencia de algunos compradores.</p>", unsafe_allow_html=True)
        if input_data['Hospital'] == 1:
            st.subheader("Hospital")
            st.markdown("<p style='text-align: justify;'>La vivienda está cerca de un hospital, lo cual puede ser valorado positivamente por la accesibilidad a servicios médicos. "
                        "La proximidad a un hospital es una característica valorada por muchas familias, especialmente aquellas que buscan tener atención médica cercana en caso de emergencia.</p>", unsafe_allow_html=True)
        if input_data['Metro'] == 1:
            st.subheader("Metro")
            st.markdown("<p style='text-align: justify;'>La vivienda está cerca de una estación de metro, lo que aumenta su atractivo debido a la facilidad de transporte. "
                        "El acceso fácil al metro es una ventaja para aquellos que se desplazan frecuentemente por la ciudad, ya que reduce los tiempos de viaje y mejora la conectividad con otras áreas.</p>", unsafe_allow_html=True)

    else:
        # Muestra un mensaje de advertencia si los datos necesarios no están disponibles
        st.warning("Primero debes calcular el valor estimado de la vivienda.")
    if st.button("Volver"):
        st.session_state.selected_tab = "Estimador real"  # Redirige al estimador si es necesario


Overwriting ExplicacionVariablesModelo.py


In [42]:
%%writefile Modelo.py

# Importación de la librería Streamlit
import streamlit as st

# Función principal que genera la página de explicación del modelo
def mostrar_explicacion_modelo():
    # Título de la página en Streamlit
    st.title("Explicación del Modelo de Estimación de Viviendas")

    # Subtítulo para la sección de detalles del modelo
    st.header("Detalles del Modelo")
    st.write("""
   El modelo utilizado para estimar el valor de las viviendas se basa en un algoritmo de llamado **Random Forest Regresion**.
    Este tipo de modelo predice el valor de una vivienda basándose en diferentes características, como el tamaño, el
    número de habitaciones y la ubicación.
    """)

    # Creación de un contenedor para dividir la interfaz en secciones
    with st.container():
        # Título para la sección de características clave
        st.header("Características clave")

        # División de la sección en dos columnas
        Datos_Usados, Precisión_Métricas = st.columns((1, 1))

        # Columna para explicar los datos utilizados para el entrenamiento del modelo
        with Datos_Usados:
            st.header("Datos Usados")
            st.markdown(
                """
                <div class="justify">
                El modelo se entrenó utilizando un conjunto de datos de ventas históricas de viviendas del condado de kings en EEUU. Estos incluyen variables como:

                - **Tamaño** de la vivienda en metros cuadrados.
                - **Número de habitaciones**.
                - **Número de baños**
                - **Ubicación** (barrio o ciudad).
                - **Año de construcción**.
                - **Estado** de la vivienda.
                - Entre otras.
                </div>
                """, unsafe_allow_html=True)  # Se permite usar HTML para dar formato

        # Columna para explicar las métricas de precisión del modelo
        with Precisión_Métricas:
            st.header("Precisión y Métricas")
            st.markdown(
                """
                <div class="justify">
                El rendimiento del modelo se mide utilizando varias métricas de evaluación, tales como:

                - **Error Cuadrático Medio (MSE)**: Esta métrica mide el promedio de los errores al cuadrado entre los valores predichos y los reales.
                - **R² (Coeficiente de Determinación)**: Indica qué tan bien los datos se ajustan al modelo; un valor cercano a 1 indica un buen ajuste.

                El modelo ha sido entrenado y probado con estos datos para asegurar una buena precisión en las estimaciones.
                </div>
                """, unsafe_allow_html=True)  # Se permite usar HTML para dar formato

    # Título para la sección del video de introducción
    st.title("Video de Introducción")
    # Se agrega un video de YouTube con un enlace específico
    youtube_url = "https://www.youtube.com/watch?v=NZ4UaZBNIRs"
    st.video(youtube_url)

# Ejecuta la función para mostrar la página solo cuando el script se ejecuta directamente
if __name__ == "__main__":
  mostrar_explicacion_modelo()


Writing Modelo.py


In [43]:
%%writefile FAQ.py

# Importación de la librería Streamlit
import streamlit as st

# Función que genera la página de Preguntas Frecuentes (FAQ)
def mostrar_faq():
    # Título de la página de Preguntas Frecuentes
    st.title("Preguntas Frecuentes (FAQ)")

    # Expansión de la Pregunta 1 sobre el modelo de estimación de viviendas
    with st.expander("¿Qué es un modelo de estimación de viviendas?"):
        st.markdown("""
        <div style='text-align: justify;'>
        Un modelo de estimación de viviendas es una herramienta que utiliza algoritmos y datos históricos para predecir
        el valor de una propiedad basándose en diversas características, como el tamaño, la ubicación, el número de
        habitaciones, entre otros.
        </div>
        """, unsafe_allow_html=True)  # Se permite el uso de HTML para personalizar el estilo

    # Expansión de la Pregunta 2 sobre los datos utilizados por el modelo
    with st.expander("¿Qué tipo de datos utiliza el modelo?"):
        st.markdown("""
        <div style='text-align: justify;'>
        El modelo utiliza datos de propiedades, tales como:

        - Tamaño en metros cuadrados

        - Número de habitaciones y baños

        - Año de construcción

        - Ubicación (ciudad, barrio)

        - Estado de la propiedad

        </div>
        """, unsafe_allow_html=True)  # Se permite el uso de HTML para personalizar el estilo

    # Expansión de la Pregunta 3 sobre la precisión del modelo
    with st.expander("¿Cuán preciso es el modelo?"):
        st.markdown("""
        <div style='text-align: justify;'>
        La precisión del modelo depende de la cantidad y calidad de los datos utilizados. Sin embargo, nuestro modelo
        se entrena con grandes cantidades de datos históricos, alcanzando una precisión del 85% en las estimaciones.
        </div>
        """, unsafe_allow_html=True)  # Se permite el uso de HTML para personalizar el estilo

    # Expansión de la Pregunta 4 sobre si confiar en las estimaciones para la compra
    with st.expander("¿Puedo confiar en las estimaciones para una compra?"):
        st.markdown("""
        <div style='text-align: justify;'>
        El modelo proporciona estimaciones basadas en datos históricos, pero siempre es recomendable consultar a un
        profesional inmobiliario para decisiones de compra importantes, ya que el valor de una propiedad puede verse
        afectado por factores adicionales no contemplados en los datos.
        </div>
        """, unsafe_allow_html=True)  # Se permite el uso de HTML para personalizar el estilo

    # Expansión de la Pregunta 5 sobre qué hacer si los resultados no parecen precisos
    with st.expander("¿Qué debo hacer si los resultados no parecen precisos?"):
        st.markdown("""
        <div style='text-align: justify;'>
        Si consideras que los resultados no son precisos, es posible que algunos de los datos ingresados (como tamaño o
        ubicación) no reflejen correctamente las características de la propiedad. También puedes intentar ajustar los
        valores o actualizar los datos con información más reciente.
        </div>
        """, unsafe_allow_html=True)  # Se permite el uso de HTML para personalizar el estilo

    # Expansión de la Pregunta 6 sobre qué es la ciencia de datos
    with st.expander("¿Qué es ciencia de datos?"):
        st.markdown("""
        <div style='text-align: justify;'>
        La ciencia de datos es un campo que combina técnicas estadísticas, matemáticas y computacionales para analizar
        grandes volúmenes de datos y extraer información útil. Se utiliza para tomar decisiones informadas basadas en
        los patrones y conocimientos extraídos de los datos.
        </div>
        """, unsafe_allow_html=True)  # Se permite el uso de HTML para personalizar el estilo

    # Expansión de la Pregunta 7 sobre qué es machine learning
    with st.expander("¿Qué es machine learning?"):
        st.markdown("""
        <div style='text-align: justify;'>
        El machine learning (aprendizaje automático) es una rama de la inteligencia artificial que permite a los
        algoritmos aprender y mejorar automáticamente a partir de la experiencia, sin ser explícitamente programados.
        Esto permite realizar predicciones o decisiones basadas en datos.
        </div>
        """, unsafe_allow_html=True)  # Se permite el uso de HTML para personalizar el estilo

# Esta línea asegura que la función se ejecute solo cuando el archivo se ejecute directamente
if __name__ == "__main__":
    mostrar_faq()



Writing FAQ.py


Los siguientes codigos son para generar el entorno donde se ejecuta la pagina web

In [35]:
!streamlit run app.py --server.port 8501 &>/dev/null &

In [36]:
!pip install pyngrok

In [37]:
#Quitar el comentario si es que se tienen muchos tueneles abiertos y la pagina tira error
ngrok.kill() #Cierra los tuneles en caso de que existan muchos

In [38]:
from pyngrok import ngrok
ngrok.set_auth_token("2oGj2rSv7bt2w7cd7RlLfKK3OoR_VhkNyq9Nr4fUvJerwWgi")

In [40]:
from pyngrok import ngrok
public_url = ngrok.connect(8501, "http")  # Define el puerto sin usar una cadena
print(f"Open this URL to access your app: {public_url}")

Open this URL to access your app: NgrokTunnel: "https://da5d-34-86-190-7.ngrok-free.app" -> "http://localhost:8501"
